In [ ]:
###########################################################
#
# Sequential Minimal Optimisation ( SMO ) - Kernel Dual SVM
#
############################################################
#
# Dual SVM - max W(alpha) = sum(alpha_i, i=1 to n) - 0.5 * sum(sum(alpha_i * alpha_j * y_i * y_j * K(x_i, x_j), j=1 to n), i=1 to n)
#
# subject to:
# 0 <= alpha_i <= C  for all i
# sum(alpha_i * y_i, i=1 to n) = 0
# SMO exists to solve this efficiently - which works by updating two Lagrange multiplier at the same time
# Previous way of updating one alpha at a time broke te sum(alpha_i*y_i) = 0 condition as it didnt check this condition at all 
# Now we update alpha_i and choose another alpha_j to compensate the change and move towards an optimal solution with KKT conditions valid and the above as well
# alpha_i*y_i + alpha_j*y_j = constant ( both alphas constraint to [0,C])
# W(alpha_i, alpha_j) = alpha_i + alpha_j - 0.5 * (K_ii * alpha_i^2 + K_jj * alpha_j^2 + 2 * y_i * y_j * K_ij * alpha_i * alpha_j) + constant
# This subproblem has a closed form solution
# E_i = f(x_i) - y_i
# f(x) = sum(alpha_k * y_k * K(x_k, x), k) + b
# Maximizing the 1D quadratic gives us the new alpha_j value which we clip to [L,H] (these constraints come from the constraint on alpha_j and alpha_i remaining in a range and satisfying the initial condition)
# Calculate new alpha_i from alpha_j
# Similarly new b1 and b2 are obtained
# Checking function is KKT 


In [14]:
import numpy as np

# Kernels  - linear, polynomial, rbf

def linear_kernel(x,z):
    return (np.dot(x,z))

def polynomial_kernel(x,z,degree=2,coeff=0.0):
    return (np.dot(x,z)+coeff)**degree

def rbf_kernel(x,z,sigma=1.0):
    return np.exp(-np.linalg.norm(x-z)**2/(2*sigma**2))

def kernel_matrix(X,kernel):
    n=X.shape[0]
    K=np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            K[i,j] = kernel(X[i],X[j])
    return K

def dual_svm_kernel_smo(X,Y,kernel,C=1.0,tol=1e-3,m_cycles=20000):
    
    n_samples,n_features = X.shape
    alpha = np.zeros(n_samples)
    b = 0.0
    K = kernel_matrix(X,kernel)
    def error(i):
        return np.sum(alpha*Y*K[:,i]) + b - Y[i]

    def violating_kkt(i,E_i):
        r = E_i * Y[i]
        return (r < -tol and alpha[i] < C) or (r > tol and alpha[i] > 0)     

    cycles = 0
    while cycles<m_cycles:
        n_changed = 0

        for i in range(n_samples):
            E_i = error(i)
            if not violating_kkt(i,E_i):
                continue

            errors = np.array([error(k) for k in range(n_samples)])
            error_diffs = np.abs(errors - E_i)
            error_diffs[i] = -np.inf
            j = np.argmax(error_diffs)
            E_j = errors[j]
            alpha_i_old = alpha[i]
            alpha_j_old = alpha[j]
            
            if Y[i] != Y[j]:
                L = max(0, alpha[j] - alpha[i])
                H = min(C, C + alpha[j] - alpha[i])
            else:
                L = max(0, alpha[i] + alpha[j] - C)
                H = min(C, alpha[i] + alpha[j])
    
            if L == H:
                continue
            eta = K[i, i] + K[j, j] - 2 * K[i, j]
            if eta <= 0:
                continue

            alpha[j] = alpha_j_old + Y[j] * (E_i - E_j) / eta
            alpha[j] = np.clip(alpha[j], L, H)
            
            if abs(alpha[j] - alpha_j_old) < 1e-5:
                continue
            alpha[i] = alpha_i_old + Y[i] * Y[j] * (alpha_j_old - alpha[j])

            b1 = b - E_i - Y[i] * K[i, i] * (alpha[i] - alpha_i_old) - Y[j] * K[i, j] * (alpha[j] - alpha_j_old)
            b2 = b - E_j - Y[i] * K[i, j] * (alpha[i] - alpha_i_old) - Y[j] * K[j, j] * (alpha[j] - alpha_j_old)
            if 0 < alpha[i] < C:
                b = b1
            elif 0 < alpha[j] < C:
                b = b2
            else:
                b = (b1 + b2) / 2
            
            n_changed += 1

        if n_changed == 0:
            cycles += 1
        else:
            cycles = 0
    
    return alpha, b
# Training produces alpha, b -> the support vectors and we use kernel to see how similar the new point is to the SV's
def decision_function(X_train,Y,alpha,b,kernel,x):
    sv_idx = alpha > 1e-6
    return np.sum(alpha[sv_idx] * Y[sv_idx] *np.array([kernel(xi,x) for xi in X_train[sv_idx]]))+b


In [15]:
def dual_svm_kernel(X,Y,kernel,C=None,step_size=1e-3,cycles=20000):

    n_samples=X.shape[0]
    alpha=np.zeros(n_samples)
    b=0.0
    K=kernel_matrix(X,kernel)

    for _ in range(cycles):
        for i in range(n_samples):
            margin = Y[i] * (np.sum(alpha* Y * K[:,i])+b)
            if margin<1:
                alpha[i]+=step_size*(1-margin)
                if C is not None:
                    alpha[i]=np.clip(alpha[i],0,C)
                else:
                    alpha[i]=max(alpha[i],0)

        sv=alpha>1e-6
        if np.any(sv):
            b=np.mean(Y[sv]-np.sum(alpha*Y*K[:,sv],axis=0))

    return alpha,b

In [16]:
X = np.array([
    [1.0, 1.0], [1.2, 0.8], [0.8, 1.3], [1.1, 1.4],
    [3.0, 3.1], [2.8, 2.9], [3.2, 2.7], [3.1, 3.3]
])

Y = np.array([-1, -1, -1, -1, 1, 1, 1, 1])

print("================= GRADIENT ASCENT ====================")
alpha_h, b_h = dual_svm_kernel(X, Y, linear_kernel)
check_h = np.array([Y[i] * decision_function(X, Y, alpha_h, b_h, linear_kernel, X[i]) for i in range(len(X))])
print(f"b : {b_h:.4f}")
print(f"alpha : {alpha_h}")
print(f"check : {check_h}")

print("\n================= SMO ====================")
alpha_smo, b_smo = dual_svm_kernel_smo(X, Y, linear_kernel, C=1.0)
check_smo = np.array([Y[i] * decision_function(X, Y, alpha_smo, b_smo, linear_kernel, X[i]) for i in range(len(X))])
print(f"b : {b_smo:.4f}")
print(f"alpha : {alpha_smo}")
print(f"check : {check_smo}")


================= GRADIENT ASCENT ====================
b : -2.5376
alpha : [0.05415677 0.05498561 0.059014   0.25710127 0.06070088 0.20068304
 0.07499278 0.04230803]
check : [1.29514513 1.27763723 1.2549085  0.99766852 1.24745975 0.99897372
 1.14947859 1.42944731]

================= SMO ====================
b : -2.1698
alpha : [0.23781213 0.         0.03806444 0.         0.23781213 0.03806444
 0.         0.        ]
check : [1.05774194 1.05945303 1.         0.7784432  1.22241234 1.
 1.10863954 1.38964937]


In [17]:
X = np.array([
    [0.0, 0.0],
    [0.0, 1.0],
    [1.0, 0.0],
    [1.0, 1.0]
])

Y = np.array([-1, 1, 1, -1])

print("\n================= XOR: GRADIENT ASCENT ====================")
alpha_s, b_s = dual_svm_kernel(X, Y, rbf_kernel, C=1.0)
check_s = np.array([Y[i] * decision_function(X, Y, alpha_s, b_s, rbf_kernel, X[i])for i in range(len(X))])
print(f"b : {b_s:.4f}")
print(f"alpha : {alpha_s}")
print(f"check : {check_s}")

print("\n================= XOR: SMO ====================")
alpha_smo, b_smo = dual_svm_kernel_smo(X, Y, rbf_kernel, C=1.0)
check_smo = np.array([Y[i] * decision_function(X, Y, alpha_smo, b_smo, rbf_kernel, X[i]) for i in range(len(X))])
print(f"b : {b_smo:.4f}")
print(f"alpha : {alpha_smo}")
print(f"check : {check_smo}")



================= XOR: GRADIENT ASCENT ====================
b : 0.0000
alpha : [1. 1. 1. 1.]
check : [0.15481812 0.15481812 0.15481812 0.15481812]

================= XOR: SMO ====================
b : 0.0000
alpha : [1. 1. 1. 1.]
check : [0.15481812 0.15481812 0.15481812 0.15481812]
